# Breast cancer
---

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from sklearn import datasets

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
breast_cancer = datasets.load_breast_cancer(as_frame=True)
print(breast_cancer.DESCR)

In [ ]:
breast_cancer.data.corr()

In [ ]:
X = breast_cancer.data.values
y = breast_cancer.target.values
m, n = X.shape
print(f"{X.shape=}")
print(f"{y.shape=}")
print(f"{m=}")
print(f"{n=}")

In [ ]:
y = y.reshape(-1, 1)
print(f"{y.shape=}")

In [ ]:
np.random.seed(0)
w = np.random.randn(1, n)
b = np.random.randn(1)

In [ ]:
def linear_combination(w, b, X):
    return X @ w.T + b

def model(w, b, X):
    return sigmoid(linear_combination(w, b, X))

The values resulting from the matrix multiplication are huge and when they get inputed into the sigmoid, they saturate:

In [ ]:
model(w, b, X)[:10]

We know that the weights are drawn from a normal distribution, thus, they have mean of 0 and standard deviation of 1. Despite the weights being close to 0, the linear combination outputs huge values, therefore it is because the input features are already very huge which we can check by examining the mean of each feature:

In [ ]:
breast_cancer.data.describe().loc["mean"]

**We need to normalize our input features.**

In [ ]:
X = (X - X.mean(axis=0, keepdims=True)) / X.std(axis=0, keepdims=True)

In [ ]:
(X @ w.T)[:10]

In [ ]:
model(w, b, X)[:10]

Much better, the model will be able to learn.

In [ ]:
output = model(w, b, X)
nll = - sum(y * np.log(output) + (1-y) * np.log(1-output)) / m
print(f"negative log likelihood = {nll.item():.3f}")

In [ ]:
linear = nn.Linear(n, 1)
linear.weight = nn.Parameter(torch.from_numpy(w))
linear.bias = nn.Parameter(torch.from_numpy(b))

output_ = torch.sigmoid(linear(torch.from_numpy(X)))

loss = F.binary_cross_entropy(output_, torch.from_numpy(y.astype(np.float64)))
print(f"nll from pytorch: {loss.item():.3f}")

In [ ]:
np.random.seed(0)
w = np.random.randn(1, n)
b = np.random.randn(1)

losses = []
for i in range(300):
    output = model(w, b, X)
    nll = - sum(y * np.log(output) + (1-y) * np.log(1-output)) / m

    dw = (1/m) * np.dot(X.T, (output - y))
    db = (1/m) * np.sum(output - y)

    w -= .1 * dw.T
    b -= .1 * db.T
    losses.append(nll)

pd.DataFrame(losses).plot();